# DECISION TREE Y RANDOM FOREST
---

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('once')

In [2]:
df_est = pd.read_csv("datos/05-bikes_encoding_est.csv", index_col = 0)
df_est.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,registered
0,0,0,0,1,0,0,1,-0.691698,1.254842,-0.360109,654
1,0,0,1,0,2,1,1,-0.752316,0.465708,0.867064,670
2,0,0,1,0,2,1,2,-1.759976,-1.395307,0.863860,1229
3,1,0,1,0,0,0,2,-1.620681,-0.293988,-0.362199,1454
4,1,0,2,0,0,1,2,-1.515533,-1.397579,0.008406,1518


In [3]:
df_est.season.unique()

array([0, 1, 2], dtype=int64)

In [4]:
df_est.isnull().sum()

season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
atemp         0
hum           0
windspeed     0
registered    0
dtype: int64

In [20]:
# Creamos una clase para poder hacer el decision tree y el random forest más rápido
class Regres_lineal:
    def __init__(self, dataf, col_pred):
        
        self.dataf = dataf
        self.col_pred = col_pred

    def tree_param(self):
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

        arbol = DecisionTreeRegressor(random_state =0)
    
        arbol.fit(x_train, y_train)

        max_feat = np.sqrt(len(x_train.columns))
        max_dep = arbol.tree_.max_depth
        return(max_feat, max_dep)

    def metricas(self, y_te, y_tr, y_te_pred, y_tr_pred, tipo_modelo):
        resultados = {'MAE': [mean_absolute_error(y_te, y_te_pred), mean_absolute_error(y_tr, y_tr_pred)],
                    'MSE': [mean_squared_error(y_te, y_te_pred), mean_squared_error(y_tr, y_tr_pred)],
                    'RMSE': [np.sqrt(mean_squared_error(y_te, y_te_pred)), np.sqrt(mean_squared_error(y_tr, y_tr_pred))],
                    'R2':  [r2_score(y_te, y_te_pred), r2_score(y_tr, y_tr_pred)],
                    "set": ["test", "train"]}
        df = pd.DataFrame(resultados)
        df["modelo"] = tipo_modelo
        return df

    def grid_s_decision(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= GradientBoostingRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

    def grid_s_forest(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= RandomForestRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

In [21]:

rg_est = Regres_lineal(df_est, "registered")

In [22]:
features_est, depth_est = rg_est.tree_param()

In [23]:
features_est #El número máx será 3

3.1622776601683795

In [24]:
depth_est # La profundidad máxima será 22

22

In [25]:
param1 = {"max_depth": [16, 18, 20], 
        "max_features": [3,4],
        "min_samples_split": [50, 100],
        "min_samples_leaf": [50,100]} 

In [28]:
best_model1, result1 = rg_est.grid_s_decision(param1, "GradientBoosting Estand I")

In [29]:
best_model1

GradientBoostingRegressor(max_depth=20, max_features=4, min_samples_leaf=50,
                          min_samples_split=100)

In [30]:
result1

,MAE,MSE,RMSE,R2,set,modelo
0,568.414695,524670.752318,724.341599,0.779613,test,GradientBoosting Estand I
1,473.720407,374040.136611,611.588208,0.846057,train,GradientBoosting Estand I


In [31]:
param2 = {"max_depth": [4,6,8,10, 12, 14, 16, 18, 20], 
        "max_features": [2, 3,4],
        "min_samples_split": [10, 20, 40, 50, 60],
        "min_samples_leaf": [10, 20, 40, 50, 60]} 

In [15]:
best_model2, result2 = rg_est.grid_s_decision(param2, "GradientBoosting Estand II")

In [32]:
best_model2

GradientBoostingRegressor(max_depth=4, max_features=3, min_samples_leaf=10,
                          min_samples_split=20)

In [33]:
result2

,MAE,MSE,RMSE,R2,set,modelo
0,554.547429,500720.682542,707.616197,0.789674,test,GradientBoosting Estand II
1,400.906322,273713.727474,523.176574,0.887348,train,GradientBoosting Estand II


In [75]:
param3 = {"max_depth": [6, 8, 10, 12, 14, 16, 18], 
        "max_features": [3,4],
        "min_samples_split": [10, 20, 40],
        "min_samples_leaf": [10, 20, 40]} 

In [76]:
best_model3, result3 = rg_est.grid_s_forest(param3, "RandForest Estand I")

In [77]:
best_model3

DecisionTreeRegressor(max_depth=16, max_features=4, min_samples_leaf=10,
                      min_samples_split=20)

In [78]:
result3

,MAE,MSE,RMSE,R2,set,modelo
0,641.022819,704907.00082,839.587399,0.703906,test,RandForest Estand I
1,563.570129,544300.78273,737.767431,0.775983,train,RandForest Estand I


In [79]:
param4 = {"max_depth": [12, 14, 16, 18], 
        "max_features": [4,5],
        "min_samples_split": [10, 20, 40, 50],
        "min_samples_leaf": [10, 20, 40, 50]} 

In [80]:
best_model4, result4 = rg_est.grid_s_forest(param4, "RandForest Estand II")

In [81]:
best_model4

DecisionTreeRegressor(max_depth=16, max_features=5, min_samples_leaf=10,
                      min_samples_split=20)

In [82]:
result4

,MAE,MSE,RMSE,R2,set,modelo
0,668.949818,711515.960498,843.514055,0.701130,test,RandForest Estand II
1,541.686623,515975.047631,718.314031,0.787641,train,RandForest Estand II


In [83]:
df_sinest = pd.read_csv("datos/04-bikes_encoding_sinest.csv", index_col = 0)
df_sinest.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,registered
0,0,0,0,0,0,0,2,18.18125,80.5833,10.749882,654
1,0,0,1,1,2,1,2,17.68695,69.6087,16.652113,670
2,0,0,1,1,2,1,3,9.47025,43.7273,16.636703,1229
3,2,0,1,1,0,0,3,10.60610,59.0435,10.739832,1454
4,2,0,2,1,0,1,3,11.46350,43.6957,12.522300,1518


In [84]:
rg_sinest = Regres_lineal(df_sinest, "registered")

In [85]:
features_sinest, depth_sinest = rg_sinest.tree_param()

In [86]:
features_sinest #El número máx será 3

3.1622776601683795

In [87]:
depth_sinest # La profundidad máxima será 22

22

In [88]:
best_model5, result5 = rg_sinest.grid_s_decision(param1, "GradientBoosting SinEstand I")

In [89]:
result5

,MAE,MSE,RMSE,R2,set,modelo
0,825.387916,1.104569e+06,1050.984965,0.536029,test,DecTree SinEstand I
1,778.572830,9.618403e+05,980.734579,0.604137,train,DecTree SinEstand I


In [90]:
best_model6, result6 = rg_sinest.grid_s_decision(param2, "GradientBoosting SinEstand II")

In [91]:
result6

,MAE,MSE,RMSE,R2,set,modelo
0,753.378527,859894.571768,927.305005,0.638804,test,DecTree SinEstand II
1,595.433737,587833.294244,766.702872,0.758067,train,DecTree SinEstand II


In [92]:
best_model7, result7 = rg_sinest.grid_s_forest(param3, "RandForest SinEstand I")

In [93]:
result7

,MAE,MSE,RMSE,R2,set,modelo
0,740.636828,929141.620332,963.919924,0.609717,test,RandForest SinEstand I
1,623.349686,715395.653498,845.810649,0.705566,train,RandForest SinEstand I


In [94]:
best_model8, result8 = rg_sinest.grid_s_forest(param4, "RandForest SinEstand II")

In [95]:
result8

,MAE,MSE,RMSE,R2,set,modelo
0,896.018135,1.336578e+06,1156.104458,0.438574,test,RandForest SinEstand II
1,686.058488,8.136515e+05,902.026319,0.665127,train,RandForest SinEstand II


In [96]:
param5 = {"max_depth": [22], 
        "max_features": [4],
        "min_samples_split": [8],
        "min_samples_leaf": [8]}

In [97]:
best_model9, result9 = rg_sinest.grid_s_forest(param5, "RandForest SinEstand III")

In [98]:
result9

,MAE,MSE,RMSE,R2,set,modelo
0,749.410366,915804.470349,956.976734,0.615319,test,RandForest SinEstand III
1,569.126633,556333.706007,745.877809,0.771031,train,RandForest SinEstand III


In [99]:
best_model10, result10 = rg_est.grid_s_forest(param5, "RandForest Estand III")

In [100]:
result10

,MAE,MSE,RMSE,R2,set,modelo
0,665.095637,702574.828438,838.197368,0.704885,test,RandForest Estand III
1,536.185702,497074.323463,705.034980,0.795420,train,RandForest Estand III


In [101]:
param6 = {"max_depth": [18,20], 
        "max_features": [3,4],
        "min_samples_split": [8, 15],
        "min_samples_leaf": [8, 15]}

In [102]:
best_model11, result11 = rg_est.grid_s_forest(param6, "RandForest Estand IV")

In [103]:
best_model11

DecisionTreeRegressor(max_depth=20, max_features=4, min_samples_leaf=8,
                      min_samples_split=15)

In [104]:
result11

,MAE,MSE,RMSE,R2,set,modelo
0,760.044417,967845.421681,983.791351,0.593459,test,RandForest Estand IV
1,560.851412,533750.181460,730.582084,0.780326,train,RandForest Estand IV


In [105]:
best_model12, result12 = rg_est.grid_s_decision(param5, "GradientBoosting Estand III")

In [106]:
result12

,MAE,MSE,RMSE,R2,set,modelo
0,659.998641,802307.138520,895.715992,0.662993,test,Decision Estand III
1,569.499281,545561.348785,738.621249,0.775464,train,Decision Estand III


In [107]:
best_model13, result13 = rg_sinest.grid_s_decision(param5, "GradientBoosting SinEstand III")

In [108]:
result13

,MAE,MSE,RMSE,R2,set,modelo
0,763.273980,979329.525348,989.610795,0.588635,test,Decision SinEstand III
1,560.295604,535269.917733,731.621431,0.779700,train,Decision SinEstand III


In [109]:
param7 = {"max_depth": [18,20], 
        "max_features": [4],
        "min_samples_split": [8, 16],
        "min_samples_leaf": [8]}

In [110]:
best_model14, result14 = rg_est.grid_s_decision(param7, "GradientBoosting Estand IV")

In [111]:
best_model14

DecisionTreeRegressor(max_depth=20, max_features=4, min_samples_leaf=8,
                      min_samples_split=8)

In [112]:
result14

,MAE,MSE,RMSE,R2,set,modelo
0,665.789526,711869.192670,843.723410,0.700981,test,Decision Estand IV
1,564.926909,550413.797009,741.898778,0.773467,train,Decision Estand IV
